In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May 20 14:49:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1050    Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   53C    P5    N/A /  N/A |    401MiB /  4040MiB |      3%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os

In [6]:
class Config:
    DATASET_PATH ="/PATH_TO_DATASET/SPID/Train"
    DATASET_PATH2 ="/PATH_TO_DATASET/SPID/Test"
    GT_PATH = "/PATH_TO_DATASET/SPID/Train_GT/"
    GT_PATH2 = "/PATH_TO_DATASET/SPID/Test_GT/"

In [4]:
def gt_path():
    all_gt = []

    for f in sorted(listdir(Config.GT_PATH)):
        if isdir(join(Config.GT_PATH, f)):
            gtl = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(join(Config.GT_PATH, f))):
                if str(join(join(Config.GT_PATH, f), c))[-3:] == "xml":
                    gt_path = (join(join(Config.GT_PATH, f), c))
                    #img = np.array(img, dtype=np.float32) / 256.0
                    gtl.append(gt_path)
            all_gt.extend(gtl)
            # get the 10-frames sequences from the list of images after applying data augmentation
    
    return all_gt


def path_strides():
    clips = []

    for f in sorted(listdir(Config.DATASET_PATH)):
        if isdir(join(Config.DATASET_PATH, f)):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(join(Config.DATASET_PATH, f))):
                if str(join(join(Config.DATASET_PATH, f), c))[-3:] == "jpg":
                    img_path = (join(join(Config.DATASET_PATH, f), c))
                    #img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img_path)
            
            clips.extend(all_frames)
    return clips
    
from os import listdir
from os.path import isfile, join, isdir
from PIL import Image
import numpy as np
import shelve

In [5]:
img_paths_all = path_strides()

groundtruth_paths = gt_path()

steps_tmp = len(img_paths_all)

print(steps_tmp)

gt_tmp = len(groundtruth_paths)

print(gt_tmp)


14550
14420


In [ ]:
gt_img_names = []

for i in groundtruth_paths:
    a = os.path.basename(i)
    b = os.path.splitext(a)[0]
    print(b)
    gt_img_names.append(b)

print(len(gt_img_names))

In [7]:
import re

img_paths_gt_only = []

for i in gt_img_names:
    res = [x for x in img_paths_all if re.search(i, x)] 
    img_paths_gt_only.append(res)
    
print(len(img_paths_gt_only))

14420


In [ ]:
print(img_paths_gt_only)

In [9]:
print(gt_img_names)

['BridgeEntrance-001_00030', 'BridgeEntrance-001_00054', 'BridgeEntrance-001_00071', 'BridgeEntrance-001_00126', 'BridgeEntrance-001_00163', 'BridgeEntrance-001_00213', 'BridgeEntrance-001_00235', 'BridgeEntrance-001_00260', 'BridgeEntrance-001_00287', 'BridgeEntrance-001_00297', 'BridgeEntrance-001_00340', 'BridgeEntrance-001_00374', 'BridgeEntrance-001_00389', 'BridgeEntrance-001_00400', 'BridgeEntrance-001_00417', 'BridgeEntrance-002_00008', 'BridgeEntrance-002_00613', 'BridgeEntrance-003_00015', 'BridgeEntrance-003_00075', 'BridgeEntrance-003_00127', 'BridgeEntrance-003_00214', 'BridgeEntrance-003_00295', 'BridgeEntrance-003_00308', 'BridgeEntrance-004_00318', 'BridgeEntrance-004_00333', 'BridgeEntrance-004_00397', 'BridgeEntrance-004_00427', 'BridgeEntrance-004_00468', 'BridgeEntrance-004_00485', 'BridgeEntrance-004_00560', 'BridgeEntrance-004_00603', 'BridgeEntrance-004_00672', 'BridgeEntrance-004_00731', 'BridgeEntrance-004_00762', 'BridgeEntrance-005_00035', 'BridgeEntrance-005

In [ ]:
print(groundtruth_paths)

In [11]:
import shutil

In [14]:
for i in img_paths_gt_only:
    for j in i:
        shutil.copy(str(j), 'PATH_TO_DATASET/SPID/images/train')

In [57]:
from xml.etree import ElementTree
 
# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = float(box.find('xmin').text)
        ymin = float(box.find('ymin').text)
        xmax = float(box.find('xmax').text)
        ymax =float(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height

In [58]:
for i in groundtruth_paths:
    boxes, w, h = extract_boxes(i)
    # summarize extracted details
    print(boxes, w, h)

[[234.051, 130.774, 272.1647, 197.6292]] 389 423
[[257.1691, 151.3929, 285.2858, 228.2452], [277.1632, 113.904, 295.9077, 167.0133], [251.5458, 119.5273, 269.6654, 177.6352]] 389 423
[[274.0391, 110.7799, 305.2799, 171.387]] 389 423
[[280.2661, 117.4079, 303.5811, 179.3591], [257.6173, 111.4126, 282.9307, 173.3638]] 389 423
[[270.274, 123.4031, 292.9228, 180.6913]] 389 423
[[217.6488, 132.7291, 242.2961, 200.0094], [176.348, 93.4268, 191.0031, 127.4]] 389 423
[[270.9402, 146.052, 309.5764, 224.6567]] 389 423
[[276.9354, 109.4142, 298.9181, 158.0425]] 389 423
[[230.3055, 121.4047, 258.2835, 171.3654]] 389 423
[[129.7181, 112.0787, 149.7024, 157.3764]] 389 423
[[265.611, 132.7291, 294.9213, 206.6709]] 389 423
[[135.0472, 114.7433, 157.0299, 162.0394], [120.3921, 115.4094, 139.0441, 162.7055]] 389 423
[[230.3055, 134.7276, 260.948, 204.6724], [256.285, 118.074, 278.9339, 170.0331]] 389 423
[[253.6205, 114.7433, 274.937, 171.3654], [273.6047, 116.7417, 296.9197, 171.3654]] 389 423
[[23.135

[[128.0, 273.0, 164.0, 393.0], [651.0, 194.0, 725.0, 355.0]] 1920 1080
[[686.0, 182.0, 755.0, 339.0], [129.0, 244.0, 193.0, 391.0]] 1920 1080
[[685.0, 184.0, 748.0, 331.0]] 1920 1080
[[694.0, 184.0, 759.0, 330.0]] 1920 1080
[[696.0, 182.0, 758.0, 329.0], [348.0, 255.0, 376.0, 336.0]] 1920 1080
[[687.0, 180.0, 747.0, 331.0], [353.0, 202.0, 415.0, 340.0]] 1920 1080
[[688.0, 175.0, 748.0, 313.0], [387.0, 195.0, 446.0, 335.0]] 1920 1080
[[686.0, 173.0, 742.0, 298.0], [420.0, 190.0, 489.0, 331.0]] 1920 1080
[[690.0, 170.0, 741.0, 297.0], [456.0, 185.0, 508.0, 313.0]] 1920 1080
[[692.0, 164.0, 743.0, 281.0], [488.0, 176.0, 542.0, 306.0]] 1920 1080
[[691.0, 160.0, 736.0, 283.0], [516.0, 168.0, 571.0, 301.0]] 1920 1080
[[679.0, 157.0, 729.0, 275.0], [535.0, 163.0, 589.0, 285.0]] 1920 1080
[[671.0, 152.0, 725.0, 267.0], [555.0, 154.0, 611.0, 279.0]] 1920 1080
[[642.0, 148.0, 723.0, 258.0], [566.0, 148.0, 621.0, 264.0]] 1920 1080
[[646.0, 143.0, 728.0, 248.0], [576.0, 142.0, 646.0, 262.0]] 1920 

[[815.0, 152.0, 876.0, 288.0]] 1920 1080
[[799.0, 148.0, 859.0, 290.0]] 1920 1080
[[728.0, 153.0, 796.0, 288.0], [681.0, 55.0, 703.0, 102.0]] 1920 1080
[[714.0, 150.0, 775.0, 287.0]] 1920 1080
[[722.0, 138.0, 776.0, 249.0], [700.0, 46.0, 727.0, 103.0]] 1920 1080
[[710.0, 136.0, 765.0, 242.0], [686.0, 49.0, 714.0, 103.0]] 1920 1080
[[706.0, 133.0, 756.0, 235.0], [678.0, 56.0, 698.0, 103.0]] 1920 1080
[[710.0, 127.0, 754.0, 236.0], [662.0, 58.0, 682.0, 106.0]] 1920 1080
[[717.0, 120.0, 760.0, 216.0], [645.0, 56.0, 673.0, 109.0]] 1920 1080
[[721.0, 117.0, 760.0, 208.0], [631.0, 56.0, 658.0, 108.0]] 1920 1080
[[727.0, 107.0, 767.0, 202.0], [617.0, 58.0, 641.0, 109.0]] 1920 1080
[[739.0, 101.0, 782.0, 194.0], [603.0, 60.0, 631.0, 111.0]] 1920 1080
[[751.0, 91.0, 787.0, 183.0], [589.0, 60.0, 613.0, 112.0]] 1920 1080
[[754.0, 86.0, 793.0, 166.0], [576.0, 61.0, 597.0, 113.0]] 1920 1080
[[748.0, 81.0, 780.0, 161.0], [562.0, 64.0, 585.0, 116.0]] 1920 1080
[[577.0, 86.0, 609.0, 162.0], [631.0, 51

[[980.0, 340.0, 1079.0, 629.0], [1051.0, 366.0, 1161.0, 680.0], [956.0, 218.0, 1038.0, 402.0]] 1920 1080
[[904.0, 340.0, 1022.0, 631.0], [988.0, 380.0, 1138.0, 689.0], [983.0, 203.0, 1080.0, 375.0]] 1920 1080
[[812.0, 346.0, 945.0, 643.0], [968.0, 382.0, 1088.0, 700.0], [1020.0, 186.0, 1089.0, 356.0]] 1920 1080
[[681.0, 351.0, 865.0, 645.0], [914.0, 397.0, 1057.0, 734.0], [1002.0, 171.0, 1067.0, 331.0]] 1920 1080
[[643.0, 339.0, 771.0, 648.0], [824.0, 399.0, 977.0, 737.0], [974.0, 161.0, 1044.0, 315.0]] 1920 1080
[[599.0, 335.0, 716.0, 635.0], [729.0, 410.0, 899.0, 766.0], [954.0, 156.0, 1046.0, 307.0]] 1920 1080
[[546.0, 315.0, 668.0, 637.0], [636.0, 412.0, 832.0, 764.0], [946.0, 150.0, 1014.0, 304.0]] 1920 1080
[[502.0, 278.0, 623.0, 611.0], [549.0, 386.0, 753.0, 738.0], [914.0, 155.0, 999.0, 304.0]] 1920 1080
[[332.0, 279.0, 461.0, 631.0], [320.0, 322.0, 525.0, 703.0], [863.0, 143.0, 929.0, 273.0]] 1920 1080
[[206.0, 276.0, 344.0, 631.0], [144.0, 306.0, 335.0, 695.0], [863.0, 140.0,

[[1583.0, 596.0, 1744.0, 883.0]] 1920 1080
[[1771.0, 615.0, 1915.0, 898.0]] 1920 1080
[[1213.0, 300.0, 1226.0, 329.0]] 1920 1080
[[1168.0, 300.0, 1182.0, 329.0]] 1920 1080
[[1088.0, 299.0, 1100.0, 330.0]] 1920 1080
[[1144.0, 643.0, 1282.0, 962.0], [342.0, 221.0, 371.0, 295.0], [302.0, 227.0, 331.0, 323.0], [258.0, 222.0, 294.0, 330.0], [231.0, 239.0, 271.0, 340.0], [194.0, 238.0, 235.0, 357.0], [198.0, 123.0, 215.0, 163.0], [209.0, 122.0, 226.0, 162.0]] 1280 960
[[862.0, 431.0, 1015.0, 774.0], [956.0, 409.0, 1073.0, 720.0], [366.0, 235.0, 407.0, 348.0], [313.0, 241.0, 354.0, 345.0], [276.0, 233.0, 319.0, 347.0], [247.0, 249.0, 285.0, 361.0], [206.0, 249.0, 251.0, 371.0], [223.0, 121.0, 238.0, 160.0], [212.0, 123.0, 225.0, 161.0], [198.0, 124.0, 212.0, 161.0], [190.0, 124.0, 202.0, 163.0], [133.0, 154.0, 154.0, 205.0]] 1280 960
[[775.0, 322.0, 838.0, 571.0], [715.0, 328.0, 822.0, 605.0], [379.0, 250.0, 418.0, 359.0], [332.0, 249.0, 370.0, 364.0], [267.0, 257.0, 311.0, 381.0], [203.0, 12

...

[[571.0, 791.0, 860.0, 1081.0], [1367.0, 370.0, 1431.0, 499.0]] 1920 1080
[[564.0, 792.0, 859.0, 1081.0], [1316.0, 359.0, 1353.0, 487.0]] 1920 1080
[[604.0, 784.0, 821.0, 1081.0], [1234.0, 339.0, 1300.0, 472.0]] 1920 1080
[[608.0, 787.0, 822.0, 1081.0], [1173.0, 332.0, 1216.0, 441.0]] 1920 1080
[[595.0, 786.0, 821.0, 1080.0], [1109.0, 317.0, 1160.0, 428.0]] 1920 1080
[[592.0, 790.0, 820.0, 1081.0], [1088.0, 313.0, 1140.0, 414.0]] 1920 1080
[[612.0, 792.0, 820.0, 1080.0], [1098.0, 308.0, 1153.0, 406.0], [0.0, 402.0, 83.0, 665.0]] 1920 1080
[[606.0, 798.0, 821.0, 1080.0], [1154.0, 309.0, 1192.0, 394.0], [50.0, 390.0, 167.0, 664.0]] 1920 1080
[[605.0, 786.0, 823.0, 1080.0], [1199.0, 302.0, 1245.0, 385.0], [138.0, 379.0, 267.0, 636.0]] 1920 1080
[[610.0, 801.0, 824.0, 1080.0], [313.0, 358.0, 424.0, 593.0]] 1920 1080
[[458.0, 735.0, 749.0, 1080.0], [553.0, 339.0, 618.0, 524.0]] 1920 1080
[[213.0, 744.0, 473.0, 1080.0], [687.0, 336.0, 751.0, 485.0]] 1920 1080
[[473.0, 782.0, 712.0, 1080.0], 

[[709.0, 289.0, 749.0, 425.0], [1099.0, 305.0, 1141.0, 397.0], [642.0, 381.0, 712.0, 585.0], [589.0, 492.0, 696.0, 641.0], [1173.0, 305.0, 1211.0, 398.0], [559.0, 415.0, 654.0, 612.0], [1227.0, 301.0, 1252.0, 370.0]] 1920 1080
[[709.0, 289.0, 749.0, 428.0], [1112.0, 303.0, 1152.0, 399.0], [610.0, 380.0, 691.0, 550.0], [586.0, 523.0, 691.0, 648.0], [1172.0, 304.0, 1205.0, 401.0], [540.0, 442.0, 659.0, 639.0], [1228.0, 299.0, 1250.0, 356.0]] 1920 1080
[[709.0, 289.0, 749.0, 425.0], [1136.0, 303.0, 1170.0, 391.0], [583.0, 390.0, 667.0, 531.0], [572.0, 510.0, 678.0, 649.0], [1173.0, 304.0, 1203.0, 397.0], [539.0, 462.0, 650.0, 637.0], [1228.0, 299.0, 1250.0, 377.0]] 1920 1080
[[709.0, 289.0, 749.0, 425.0], [566.0, 381.0, 639.0, 594.0], [570.0, 493.0, 678.0, 631.0], [1178.0, 303.0, 1208.0, 395.0], [531.0, 441.0, 642.0, 626.0], [1227.0, 300.0, 1250.0, 392.0], [1159.0, 303.0, 1194.0, 387.0]] 1920 1080
[[709.0, 289.0, 749.0, 425.0], [557.0, 399.0, 629.0, 604.0], [603.0, 465.0, 735.0, 630.0], [

[[752.0, 291.0, 792.0, 422.0], [1214.0, 301.0, 1245.0, 394.0]] 1920 1080
[[752.0, 289.0, 793.0, 422.0], [1074.0, 302.0, 1108.0, 402.0]] 1920 1080
[[751.0, 286.0, 789.0, 423.0], [870.0, 310.0, 924.0, 437.0]] 1920 1080
[[753.0, 286.0, 791.0, 421.0], [744.0, 318.0, 785.0, 449.0]] 1920 1080
[[756.0, 292.0, 800.0, 424.0], [571.0, 327.0, 630.0, 490.0]] 1920 1080
[[753.0, 293.0, 797.0, 424.0], [526.0, 332.0, 591.0, 491.0]] 1920 1080
[[751.0, 293.0, 794.0, 424.0], [483.0, 332.0, 557.0, 492.0]] 1920 1080
[[755.0, 291.0, 797.0, 424.0], [470.0, 334.0, 550.0, 491.0]] 1920 1080
[[756.0, 289.0, 800.0, 426.0], [466.0, 338.0, 551.0, 492.0]] 1920 1080
[[757.0, 287.0, 797.0, 424.0], [478.0, 338.0, 550.0, 491.0]] 1920 1080
[[753.0, 285.0, 792.0, 424.0], [481.0, 345.0, 554.0, 492.0]] 1920 1080
[[753.0, 285.0, 795.0, 424.0], [484.0, 350.0, 557.0, 492.0]] 1920 1080
[[756.0, 285.0, 800.0, 424.0], [477.0, 355.0, 553.0, 488.0]] 1920 1080
[[757.0, 286.0, 799.0, 424.0], [474.0, 355.0, 550.0, 492.0]] 1920 1080
[[

[[584.0, 556.0, 728.0, 785.0]] 1920 1080
[[580.0, 516.0, 737.0, 784.0]] 1920 1080
[[562.0, 484.0, 693.0, 787.0]] 1920 1080
[[473.0, 481.0, 680.0, 794.0]] 1920 1080
[[447.0, 477.0, 585.0, 794.0]] 1920 1080
[[375.0, 422.0, 466.0, 695.0]] 1920 1080
[[468.0, 383.0, 574.0, 599.0]] 1920 1080
[[638.0, 361.0, 700.0, 546.0]] 1920 1080
[[777.0, 348.0, 840.0, 441.0]] 1920 1080
[[894.0, 334.0, 941.0, 478.0]] 1920 1080
[[950.0, 325.0, 998.0, 452.0]] 1920 1080
[[1090.0, 313.0, 1129.0, 416.0]] 1920 1080
[[1152.0, 311.0, 1191.0, 407.0]] 1920 1080
[[1167.0, 294.0, 1221.0, 388.0]] 1920 1080
[[1090.0, 297.0, 1145.0, 416.0]] 1920 1080
[[1022.0, 304.0, 1080.0, 429.0]] 1920 1080
[[959.0, 313.0, 1039.0, 440.0]] 1920 1080
[[905.0, 319.0, 983.0, 470.0]] 1920 1080
[[705.0, 332.0, 776.0, 508.0]] 1920 1080
[[542.0, 358.0, 653.0, 583.0]] 1920 1080
[[424.0, 387.0, 538.0, 652.0]] 1920 1080
[[249.0, 461.0, 421.0, 814.0]] 1920 1080
[[1.0, 584.0, 193.0, 1025.0]] 1920 1080
[[267.0, 375.0, 367.0, 631.0]] 1920 1080
[[581.

[[113.0161, 260.5323, 181.1452, 398.8548]] 1600 838
[[1374.4355, 359.629, 1500.371, 656.9194]] 1600 838
[[115.0806, 367.8871, 282.3065, 714.7258]] 1600 838
[[1200.0, 337.0, 1313.0, 632.0], [1300.0, 321.0, 1423.0, 625.0]] 1600 838
[[1278.1699, 290.0502, 1368.7848, 460.9239]] 1600 838
[[1115.0631, 284.8722, 1192.733, 453.157]] 1600 838
[[1128.0081, 279.6942, 1184.966, 468.6909]] 1600 838
[[292.3605, 384.6163, 450.5, 784.6163]] 1600 838
[[327.2442, 310.1977, 436.5465, 582.2907]] 1600 838
[[1166.1634, 353.7508, 1302.6942, 629.3883]] 1592 825
[[516.9984, 376.9353, 674.1375, 799.4078]] 1592 825
[[197.568, 268.7411, 261.9693, 423.3042]] 1592 825
[[1003.5988, 259.1221, 1093.843, 430.3547]] 1592 825
[[1123.9244, 275.3198, 1297.4709, 599.2733]] 1592 825
[[524.6105, 388.7035, 663.4477, 726.5407]] 1592 825
[[309.4128, 247.5523, 420.4826, 384.0756]] 1592 825
[[850.8779, 349.3663, 1005.9128, 645.5523], [952.6919, 319.2849, 1079.9593, 608.5291]] 1592 825
[[138.1802, 259.1221, 221.4826, 397.9593]] 159

In [7]:
def gt_path2():
    all_gt = []

    for f in sorted(listdir(Config.GT_PATH2)):
        if isdir(join(Config.GT_PATH2, f)):
            gtl = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(join(Config.GT_PATH2, f))):
                if str(join(join(Config.GT_PATH2, f), c))[-3:] == "xml":
                    gt_path = (join(join(Config.GT_PATH2, f), c))
                    #img = np.array(img, dtype=np.float32) / 256.0
                    gtl.append(gt_path)
            all_gt.extend(gtl)
            # get the 10-frames sequences from the list of images after applying data augmentation
    
    return all_gt


def path_strides2():
    clips = []

    for f in sorted(listdir(Config.DATASET_PATH2)):
        if isdir(join(Config.DATASET_PATH2, f)):
            all_frames = []
            # loop over all the images in the folder (0.tif,1.tif,..,199.tif)
            for c in sorted(listdir(join(Config.DATASET_PATH2, f))):
                if str(join(join(Config.DATASET_PATH2, f), c))[-3:] == "jpg":
                    img_path = (join(join(Config.DATASET_PATH2, f), c))
                    #img = np.array(img, dtype=np.float32) / 256.0
                    all_frames.append(img_path)
            clips.extend(all_frames)
    return clips
    

In [8]:
img_paths_all2 = path_strides2()

groundtruth_paths2 = gt_path2()

steps_tmp = len(img_paths_all2)

print(steps_tmp)

gt_tmp = len(groundtruth_paths2)

print(gt_tmp)


15439
15277


In [9]:
gt_img_names2 = []

for i in groundtruth_paths2:
    a = os.path.basename(i)
    b = os.path.splitext(a)[0]
    print(b)
    gt_img_names2.append(b)

print(len(gt_img_names2))

BridgeEntrance-001_00016
BridgeEntrance-001_00037
BridgeEntrance-001_00060
BridgeEntrance-001_00075
BridgeEntrance-001_00158
BridgeEntrance-001_00201
BridgeEntrance-001_00218
BridgeEntrance-001_00247
BridgeEntrance-001_00264
BridgeEntrance-001_00293
BridgeEntrance-001_00325
BridgeEntrance-001_00363
BridgeEntrance-001_00379
BridgeEntrance-001_00392
BridgeEntrance-001_00403
BridgeEntrance-002_00002
BridgeEntrance-002_00549
BridgeEntrance-002_00685
BridgeEntrance-003_00066
BridgeEntrance-003_00082
BridgeEntrance-003_00199
BridgeEntrance-003_00268
BridgeEntrance-003_00302
BridgeEntrance-004_00208
BridgeEntrance-004_00321
BridgeEntrance-004_00391
BridgeEntrance-004_00421
BridgeEntrance-004_00457
BridgeEntrance-004_00481
BridgeEntrance-004_00510
BridgeEntrance-004_00589
BridgeEntrance-004_00612
BridgeEntrance-004_00694
BridgeEntrance-004_00737
BridgeEntrance-005_00021
BridgeEntrance-005_00046
BridgeEntrance-005_00062
BridgeEntrance-005_00071
BridgeEntrance-005_00153
BridgeEntrance-005_00169


SJTU-SEIEE-094_037-02534
SJTU-SEIEE-094_037-02546
SJTU-SEIEE-094_037-02558
SJTU-SEIEE-094_037-02570
SJTU-SEIEE-094_037-02582
SJTU-SEIEE-094_037-02594
SJTU-SEIEE-094_037-02606
SJTU-SEIEE-094_037-02618
SJTU-SEIEE-094_037-02630
SJTU-SEIEE-094_037-02642
SJTU-SEIEE-094_037-02669
SJTU-SEIEE-094_037-02711
SJTU-SEIEE-094_037-02753
SJTU-SEIEE-094_037-02795
SJTU-SEIEE-094_037-02837
SJTU-SEIEE-094_037-02879
SJTU-SEIEE-094_037-02921
SJTU-SEIEE-094_037-02963
SJTU-SEIEE-094_037-03005
SJTU-SEIEE-094_037-03047
SJTU-SEIEE-094_037-03089
SJTU-SEIEE-094_037-03131
SJTU-SEIEE-094_037-03158
SJTU-SEIEE-094_037-03170
SJTU-SEIEE-094_037-03182
SJTU-SEIEE-094_037-03209
SJTU-SEIEE-094_037-03251
SJTU-SEIEE-094_037-03293
SJTU-SEIEE-094_037-03335
SJTU-SEIEE-094_037-03377
SJTU-SEIEE-094_037-03419
SJTU-SEIEE-094_037-03461
SJTU-SEIEE-094_037-03503
SJTU-SEIEE-094_037-03545
SJTU-SEIEE-094_037-03587
SJTU-SEIEE-094_037-03629
SJTU-SEIEE-094_037-03671
SJTU-SEIEE-094_037-03713
SJTU-SEIEE-094_037-03755
SJTU-SEIEE-094_037-03797


SJTU-SEIEE-094_040-05220
SJTU-SEIEE-094_040-05232
SJTU-SEIEE-094_040-05244
SJTU-SEIEE-094_040-05256
SJTU-SEIEE-094_040-05268
SJTU-SEIEE-094_040-05280
SJTU-SEIEE-094_040-05292
SJTU-SEIEE-094_040-05304
SJTU-SEIEE-094_040-05316
SJTU-SEIEE-094_040-05328
SJTU-SEIEE-094_040-05340
SJTU-SEIEE-094_040-05352
SJTU-SEIEE-094_040-05364
SJTU-SEIEE-094_040-05376
SJTU-SEIEE-094_040-05388
SJTU-SEIEE-094_040-05400
SJTU-SEIEE-094_040-05412
SJTU-SEIEE-094_040-05424
SJTU-SEIEE-094_040-05436
SJTU-SEIEE-094_040-05448
SJTU-SEIEE-094_040-05460
SJTU-SEIEE-094_040-05472
SJTU-SEIEE-094_040-05484
SJTU-SEIEE-094_040-05496
SJTU-SEIEE-094_040-05523
SJTU-SEIEE-094_040-05565
SJTU-SEIEE-094_040-05607
SJTU-SEIEE-094_040-05649
SJTU-SEIEE-094_040-05691
SJTU-SEIEE-094_040-05733
SJTU-SEIEE-094_040-05775
SJTU-SEIEE-094_040-05802
SJTU-SEIEE-094_040-05814
SJTU-SEIEE-094_040-05826
SJTU-SEIEE-094_040-05838
SJTU-SEIEE-094_040-05850
SJTU-SEIEE-094_040-05862
SJTU-SEIEE-094_040-05874
SJTU-SEIEE-094_040-05886
SJTU-SEIEE-094_040-05898


SJTU-SEIEE-136_075-03276
SJTU-SEIEE-136_075-03318
SJTU-SEIEE-136_075-03360
SJTU-SEIEE-136_075-03402
SJTU-SEIEE-136_075-03444
SJTU-SEIEE-136_075-03486
SJTU-SEIEE-136_075-03528
SJTU-SEIEE-136_075-03570
SJTU-SEIEE-136_075-03612
SJTU-SEIEE-136_075-03654
SJTU-SEIEE-136_075-03696
SJTU-SEIEE-136_075-03738
SJTU-SEIEE-136_075-03765
SJTU-SEIEE-136_075-03777
SJTU-SEIEE-136_075-03789
SJTU-SEIEE-136_075-03801
SJTU-SEIEE-136_075-03813
SJTU-SEIEE-136_075-03825
SJTU-SEIEE-136_075-03837
SJTU-SEIEE-136_075-03849
SJTU-SEIEE-136_075-03861
SJTU-SEIEE-136_075-03873
SJTU-SEIEE-136_075-03900
SJTU-SEIEE-136_075-03942
SJTU-SEIEE-136_075-03984
SJTU-SEIEE-136_075-04026
SJTU-SEIEE-136_075-04068
SJTU-SEIEE-136_075-04110
SJTU-SEIEE-136_075-04152
SJTU-SEIEE-136_075-04194
SJTU-SEIEE-136_075-04236
SJTU-SEIEE-136_075-04278
SJTU-SEIEE-136_075-04320
SJTU-SEIEE-136_075-04362
SJTU-SEIEE-136_075-04389
SJTU-SEIEE-136_075-04401
SJTU-SEIEE-136_075-04413
SJTU-SEIEE-136_075-04425
SJTU-SEIEE-136_075-04437
SJTU-SEIEE-136_075-04449


SJTU_SEIEE-182_335-07472
SJTU_SEIEE-182_335-07484
SJTU_SEIEE-182_335-07496
SJTU_SEIEE-182_335-07508
SJTU_SEIEE-182_335-07520
SJTU_SEIEE-182_335-07532
SJTU_SEIEE-182_335-07544
SJTU_SEIEE-182_335-07556
SJTU_SEIEE-182_335-07568
SJTU_SEIEE-182_335-07580
SJTU_SEIEE-182_335-07592
SJTU_SEIEE-182_335-07604
SJTU_SEIEE-182_335-08265
SJTU_SEIEE-182_335-08277
SJTU_SEIEE-182_335-08289
SJTU_SEIEE-182_335-08301
SJTU_SEIEE-182_335-08313
SJTU_SEIEE-182_335-08325
SJTU_SEIEE-182_335-08337
SJTU_SEIEE-182_335-08349
SJTU_SEIEE-182_335-08361
SJTU_SEIEE-182_335-08373
SJTU_SEIEE-182_335-08385
SJTU_SEIEE-182_335-08397
SJTU_SEIEE-182_335-08409
SJTU_SEIEE-182_335-08421
SJTU_SEIEE-182_335-08433
SJTU_SEIEE-182_335-08713
SJTU_SEIEE-182_335-08755
SJTU_SEIEE-182_335-08797
SJTU_SEIEE-182_335-08839
SJTU_SEIEE-182_335-08881
SJTU_SEIEE-182_335-08923
SJTU_SEIEE-182_335-08965
SJTU_SEIEE-182_335-08977
SJTU_SEIEE-182_335-08989
SJTU_SEIEE-182_335-09001
SJTU_SEIEE-182_335-09013
SJTU_SEIEE-182_335-09025
SJTU_SEIEE-182_335-09037


In [10]:
import re

img_paths_gt_only2 = []

for i in gt_img_names2:
    res = [x for x in img_paths_all2 if re.search(i, x)] 
    img_paths_gt_only2.append(res)
    
print(len(img_paths_gt_only2))

15277


In [ ]:
print(img_paths_gt_only2)

In [12]:
print(gt_img_names2)

['BridgeEntrance-001_00016', 'BridgeEntrance-001_00037', 'BridgeEntrance-001_00060', 'BridgeEntrance-001_00075', 'BridgeEntrance-001_00158', 'BridgeEntrance-001_00201', 'BridgeEntrance-001_00218', 'BridgeEntrance-001_00247', 'BridgeEntrance-001_00264', 'BridgeEntrance-001_00293', 'BridgeEntrance-001_00325', 'BridgeEntrance-001_00363', 'BridgeEntrance-001_00379', 'BridgeEntrance-001_00392', 'BridgeEntrance-001_00403', 'BridgeEntrance-002_00002', 'BridgeEntrance-002_00549', 'BridgeEntrance-002_00685', 'BridgeEntrance-003_00066', 'BridgeEntrance-003_00082', 'BridgeEntrance-003_00199', 'BridgeEntrance-003_00268', 'BridgeEntrance-003_00302', 'BridgeEntrance-004_00208', 'BridgeEntrance-004_00321', 'BridgeEntrance-004_00391', 'BridgeEntrance-004_00421', 'BridgeEntrance-004_00457', 'BridgeEntrance-004_00481', 'BridgeEntrance-004_00510', 'BridgeEntrance-004_00589', 'BridgeEntrance-004_00612', 'BridgeEntrance-004_00694', 'BridgeEntrance-004_00737', 'BridgeEntrance-005_00021', 'BridgeEntrance-005

In [ ]:
print(groundtruth_paths2)

In [14]:
import shutil

In [15]:
for i in img_paths_gt_only2:
    for j in i:
        shutil.copy(str(j), 'PATH_TO_DATASET/SPID/images/test')